In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('inventory.db')

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)
tables

,name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [2]:
freight_summary = pd.read_sql_query("""select VendorName, sum(Freight) as FreightCost
from vendor_invoice
group by VendorNumber""",conn)

freight_summary

,VendorName,FreightCost
0,"IRA GOLDMAN AND WILLIAMS, LLP",27.08
1,AAPER ALCOHOL & CHEMICAL CO,0.48
2,ADAMBA IMPORTS INTL INC,367.52
3,ALTAMAR BRANDS LLC,62.39
4,AMERICAN SPIRITS EXCHANGE,6.19
...,...,...
121,Serralles Usa LLC,856.02
122,STARK BREWING COMPANY,130.09
123,SWEETWATER FARM,178.34
124,TAMWORTH DISTILLING,202.50


In [3]:
pd.read_sql_query("""select 
p.VendorName,
p.VendorNumber,
p.Brand,
p.PurchasePrice,
pp.Volume, 
pp.Price as ActualPrice, 
sum(Quantity) as TotalPurchaseQuantity,
sum(Dollars) as TotalPurchasePrice
from purchases as p 
join purchase_prices as pp 
on p.Brand = pp.Brand
where p.PurchasePrice>0
group by p.VendorName,
p.VendorNumber,
p.Brand
order by TotalPurchasePrice""",conn)

,VendorName,VendorNumber,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchasePrice
0,PROXIMO SPIRITS INC.,7245,3065,0.71,50,0.99,1,0.71
1,DIAGEO NORTH AMERICA INC,3960,6127,1.47,200,1.99,1,1.47
2,HEAVEN HILL DISTILLERIES,3924,9123,0.74,50,0.99,2,1.48
3,SAZERAC CO INC,8004,5683,0.39,50,0.49,6,2.34
4,WINE GROUP INC,9815,8527,1.32,750,4.99,2,2.64
...,...,...,...,...,...,...,...,...
10687,DIAGEO NORTH AMERICA INC,3960,3545,21.89,1750,29.99,138109,3023206.01
10688,DIAGEO NORTH AMERICA INC,3960,4261,16.17,1750,22.99,201682,3261197.94
10689,PERNOD RICARD USA,17035,8068,18.24,1750,24.99,187407,3418303.68
10690,MARTIGNETTI COMPANIES,4425,3405,23.19,1750,28.99,164038,3804041.22


In [12]:
pd.read_sql_query("""select 
VendorNo, 
Brand, 
sum(SalesDollars) as TotalSalesDollars, 
sum(SalesPrice) as TotalSalesPrice, 
sum(SalesQuantity) as TotalSalesQuantity, 
sum(ExciseTax) as TotalExciseTax
from sales 
group by VendorNo, Brand
order by TotalSalesDollars""",conn)

,VendorNo,Brand,TotalSalesDollars,TotalSalesPrice,TotalSalesQuantity,TotalExciseTax
0,8004,5287,0.98,0.98,2,0.10
1,9206,2773,0.99,0.99,1,0.05
2,3252,3933,1.98,0.99,2,0.10
3,3924,9123,1.98,0.99,2,0.10
4,10050,3623,1.98,1.98,2,0.10
...,...,...,...,...,...,...
11267,3960,3545,4223107.62,545778.28,135838,249587.83
11268,3960,4261,4475972.88,420050.01,200412,368242.80
11269,17035,8068,4538120.60,461140.15,187140,343854.07
11270,4425,3405,4819073.49,561512.37,160247,294438.66


In [4]:
vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC""", conn)
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750,145080,3811251.60,142049.0,5101919.51,672819.31,260999.20,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750,164038,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750,187407,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750,201682,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750,138109,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750,2,2.64,5.0,15.95,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50,6,2.34,134.0,65.66,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50,2,1.48,2.0,1.98,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200,1,1.47,72.0,143.28,77.61,15.12,257032.07


In [5]:
vendor_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorNumber           10692 non-null  int64  
 1   VendorName             10692 non-null  object 
 2   Brand                  10692 non-null  int64  
 3   Description            10692 non-null  object 
 4   PurchasePrice          10692 non-null  float64
 5   ActualPrice            10692 non-null  float64
 6   Volume                 10692 non-null  object 
 7   TotalPurchaseQuantity  10692 non-null  int64  
 8   TotalPurchaseDollars   10692 non-null  float64
 9   TotalSalesQuantity     10514 non-null  float64
 10  TotalSalesDollars      10514 non-null  float64
 11  TotalSalesPrice        10514 non-null  float64
 12  TotalExciseTax         10514 non-null  float64
 13  FreightCost            10692 non-null  float64
dtypes: float64(8), int64(3), object(3)
memory usage: 1.1+ 

In [9]:
vendor_sales_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
PurchasePrice              0
ActualPrice                0
Volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
FreightCost                0
dtype: int64

In [10]:
vendor_sales_summary["Volume"] = vendor_sales_summary["Volume"].astype("float64")
vendor_sales_summary.fillna(0,inplace=True)
vendor_sales_summary['VendorName'] = vendor_sales_summary["VendorName"].str.strip()

In [11]:
vendor_sales_summary["GrossProfit"] = vendor_sales_summary["TotalSalesDollars"] - vendor_sales_summary["TotalPurchaseDollars"]

In [12]:
vendor_sales_summary['GrossProfit'].min()

-52002.780000000006

In [13]:
vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary["GrossProfit"]/vendor_sales_summary["TotalSalesDollars"]) *100
vendor_sales_summary['StockTurnover']  =  vendor_sales_summary["TotalSalesQuantity"]/vendor_sales_summary["TotalPurchaseQuantity"]
vendor_sales_summary["SalesPurchaseRatio"] = vendor_sales_summary["TotalSalesDollars"]/vendor_sales_summary["TotalPurchaseDollars"]

In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume ,  -- Note: The data type for Volume appears missing/incomplete in the source image
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesToPurchaseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
""")

In [16]:
vendor_sales_summary.to_sql('vendor_sales_summary',conn,if_exists= "replace", index=False)

10692

In [19]:
vendor_sales_summary.to_csv("output.csv", index=False)


In [21]:
vendor_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorNumber           10692 non-null  int64  
 1   VendorName             10692 non-null  object 
 2   Brand                  10692 non-null  int64  
 3   Description            10692 non-null  object 
 4   PurchasePrice          10692 non-null  float64
 5   ActualPrice            10692 non-null  float64
 6   Volume                 10692 non-null  float64
 7   TotalPurchaseQuantity  10692 non-null  int64  
 8   TotalPurchaseDollars   10692 non-null  float64
 9   TotalSalesQuantity     10692 non-null  float64
 10  TotalSalesDollars      10692 non-null  float64
 11  TotalSalesPrice        10692 non-null  float64
 12  TotalExciseTax         10692 non-null  float64
 13  FreightCost            10692 non-null  float64
 14  GrossProfit            10692 non-null  float64
 15  Pr